<a href="https://colab.research.google.com/github/elianoestevampuc/acuracia.io-backend/blob/main/MachineLearningMVP_Eliano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MVP da Disciplina: Sprint: Qualidade de Software, Segurança e Sistemas Inteligentes (40530010063_20240_01)

## Importação das libs

In [ ]:
import pandas as pd
from google.colab import data_table
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import json

## Classe com as rotinas para carregar os dados

In [ ]:
class Dado:

    def carregar(self, url: str, atributos: list):
        return pd.read_csv(url, names=atributos)

    def adicionar(self, url: str, dado: str):
        with open(url, 'r+') as fp:
            lines = fp.readlines()
            lines.insert(0, dado.decode() + "\n")
            fp.seek(0)
            fp.writelines(lines)

dado = Dado()
base = ('https://elianoestevampuc.github.io/acuracia.io-backend/autismo5.csv')
atributos = ['idade', 'ictericia','autismo_na_familia', 'resultado']
dataset = dado.carregar(base, atributos)
data_table.DataTable(dataset)

,idade,ictericia,autismo_na_familia,resultado
0,32,1,1,1
1,24,1,0,1
2,35,1,1,1
3,24,0,1,0
4,27,1,1,1
...,...,...,...,...
703,24,0,0,1
704,35,0,0,0
705,26,0,0,1
706,100,1,1,1


## Classe com rotinas de processamento

In [ ]:
class PreProcessador:

    def processar(self, dataset, percentual_teste, seed=7):
        # Divisão em treino e teste
        X_train, X_test, Y_train, Y_test = self.holdout(dataset,percentual_teste,seed)
        return (X_train, X_test, Y_train, Y_test)

    def holdout(self, dataset, percentual_teste, seed):
        dados = dataset.values
        X = dados[:, 0:-1]
        Y = dados[:, -1]
        return train_test_split(X, Y, test_size=percentual_teste, random_state=seed)

pre_processador = PreProcessador()
percentual_teste = 0.2
X_train, X_test, Y_train, Y_test = pre_processador.processar(dataset, percentual_teste)

## Classe modelo para treino

In [ ]:
class Modelo:

    def treinarKNN(self, X_train, Y_train):
        modelo = KNeighborsClassifier()
        modelo.fit(X_train, Y_train)
        return modelo

    def treinarSVC(self, X_train, Y_train):
        modelo = SVC()
        modelo.fit(X_train, Y_train)
        return modelo

    def treinarCART(self, X_train, Y_train):
        modelo = DecisionTreeClassifier()
        modelo.fit(X_train, Y_train)
        return modelo

    def treinarNB(self, X_train, Y_train):
        modelo = GaussianNB()
        modelo.fit(X_train, Y_train)
        return modelo

modelo = Modelo()
modelKNN = modelo.treinarKNN(X_train, Y_train)
modelSVC = modelo.treinarSVC(X_train, Y_train)
modelCART = modelo.treinarCART(X_train, Y_train)
modelNB = modelo.treinarNB(X_train, Y_train)

## Classe para cálculo e apresentação do resultado da acurácia

In [ ]:
class Acuracia:

    def avaliar(self, modelo, X_test, Y_test):
        # Predição e avaliação do modelo
        predicoes = modelo.predict(X_test)
        return accuracy_score(Y_test, predicoes)

acuracia = Acuracia()
result = json.dumps({
                                    'knn': acuracia.avaliar(modelKNN, X_test, Y_test),
                                    'svc': acuracia.avaliar(modelSVC, X_test, Y_test),
                                    'cart': acuracia.avaliar(modelCART, X_test, Y_test),
                                    'nb': acuracia.avaliar(modelNB, X_test, Y_test)
                                })
print(result)

{"knn": 0.6338028169014085, "svc": 0.7323943661971831, "cart": 0.6901408450704225, "nb": 0.6690140845070423}
